# Classification d'incidents avec des modèles *Transformers*

## 1. Création du jeu de données (*dataset*)

In [1]:
import json
import spacy
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from datasets import Dataset, DatasetDict
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
from transformers import DataCollatorWithPadding

/Users/younismbs/anaconda3/envs/myEmv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_json_fn = "./data/incidents_train.json"
validation_json_fn = "./data/incidents_test.json"
test_json_fn = "./data/incidents_test.json"

In [4]:
#Fonction permettant de charger les données
def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
    return incident_list

In [5]:
train_list = load_incident_dataset(train_json_fn)
validation_list = load_incident_dataset(validation_json_fn)
test_list = load_incident_dataset(test_json_fn)

print("Nombre d'incidents dans train:", len(train_list))
print("Nombre d'incidents dans validation:", len(validation_list))
print("Nombre d'incidents dans test:", len(test_list))

Nombre d'incidents dans train: 2475
Nombre d'incidents dans validation: 531
Nombre d'incidents dans test: 531


On normalise les données em mettant tout nos textes en minuscule avec la methode .lower():

In [6]:
train_list = [{'text': item['text'].lower(), 'label': item['label']} for item in train_list]
validation_list = [{'text': item['text'].lower(), 'label': item['label']} for item in validation_list]
test_list = [{'text': item['text'].lower(), 'label': item['label']} for item in test_list]

On convertit nos labels en int 

In [7]:
def convert_labels_to_int(dataset):
    for item in dataset:
        item['label'] = int(item['label'])

In [8]:
convert_labels_to_int(train_list)
convert_labels_to_int(validation_list)
convert_labels_to_int(test_list)

Ce code organise des données textuelles et leurs étiquettes associées en datasets séparés pour l'entraînement, la validation et le test, en utilisant la bibliothèque datasets de Hugging Face. Il crée ensuite un DatasetDict pour regrouper ces trois datasets, facilitant ainsi leur gestion et leur utilisation dans l'entraînement et l'évaluation des modèles. 

In [9]:
train_dataset = Dataset.from_dict({"text": [item['text'] for item in train_list], 
                                   "label": [item['label'] for item in train_list]})

validation_list = Dataset.from_dict({"text": [item['text'] for item in validation_list], 
                                  "label": [item['label'] for item in validation_list]})

test_dataset = Dataset.from_dict({"text": [item['text'] for item in test_list], 
                                  "label": [item['label'] for item in test_list]})

In [10]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": validation_list,
    "test": test_dataset
})

In [11]:
#On charge notre tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

On applique la tokenizeur BERT sur l'ensemble du dataset

In [13]:
tokenized_dataset = dataset_dict.map(preprocess_function, batched=True)

Map: 100%|██████████| 531/531 [00:00<00:00, 964.66 examples/s]


DataCollatorWithPadding s'occupe automatiquement de "padding" (ajout de zéros ou d'un autre token spécial) pour que tous les exemples dans un lot aient la même longueur.

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2. Création de modèle(s)

La ligne de code initialise un modèle BERT pré-entraîné pour la classification de séquences, configuré pour gérer neuf catégories distinctes, en utilisant la version "bert-base-uncased" de BERT.

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=9)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Entraînement de modèle(s)

Ce code définit une fonction compute_metrics pour évaluer l'exactitude des prédictions d'un modèle de classification en utilisant la métrique 'accuracy' de la bibliothèque Hugging Face datasets.

In [16]:
accuracy = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

Cette section de code configure un entraîneur pour notre modele en spécifiant des paramètres d'entraînement, comme le taux d'apprentissage et la taille des lots, et lie le modèle, les données d'entraînement et de test, ainsi que la méthode de calcul des métriques pour l'entraînement et l'évaluation.

In [18]:
training_args = TrainingArguments(
    output_dir="my_awesome_BERT",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

                                                    
 10%|█         | 155/1550 [23:24<2:45:43,  7.13s/it]

{'eval_loss': 0.9650126099586487, 'eval_accuracy': 0.6949152542372882, 'eval_runtime': 72.0784, 'eval_samples_per_second': 7.367, 'eval_steps_per_second': 0.472, 'epoch': 1.0}


                                                     
 20%|██        | 310/1550 [48:24<2:14:43,  6.52s/it]

{'eval_loss': 0.7241417765617371, 'eval_accuracy': 0.7608286252354048, 'eval_runtime': 79.4789, 'eval_samples_per_second': 6.681, 'eval_steps_per_second': 0.428, 'epoch': 2.0}


                                                      
 30%|███       | 465/1550 [1:13:57<1:57:02,  6.47s/it]

...

                                                     
100%|██████████| 1550/1550 [4:15:09<00:00,  6.11s/it]

{'eval_loss': 1.2029919624328613, 'eval_accuracy': 0.7589453860640302, 'eval_runtime': 78.3595, 'eval_samples_per_second': 6.776, 'eval_steps_per_second': 0.434, 'epoch': 10.0}


100%|██████████| 1550/1550 [4:15:11<00:00,  9.88s/it]

{'train_runtime': 15311.6433, 'train_samples_per_second': 1.616, 'train_steps_per_second': 0.101, 'train_loss': 0.3513666384450851, 'epoch': 10.0}


TrainOutput(global_step=1550, training_loss=0.3513666384450851, metrics={'train_runtime': 15311.6433, 'train_samples_per_second': 1.616, 'train_steps_per_second': 0.101, 'train_loss': 0.3513666384450851, 'epoch': 10.0})

In [20]:
pt_save_directory = "./pt_save_pretrainedBERT"
model.save_pretrained(pt_save_directory)

## 4. Évaluation et analyse de résultats

In [35]:
test_results = trainer.evaluate(tokenized_dataset['test'])

100%|██████████| 34/34 [01:10<00:00,  2.07s/it]


In [36]:
test_results

{'eval_loss': 0.7241417765617371,
 'eval_accuracy': 0.7608286252354048,
 'eval_runtime': 72.2949,
 'eval_samples_per_second': 7.345,
 'eval_steps_per_second': 0.47,
 'epoch': 10.0}

Les résultats affichent une précision de 76% pour notre modèle BERT, marquant la meilleure performance que nous avons réalisée jusqu'à présent. Cette précision remarquable souligne l'efficacité des modèles Transformers encodeurs, dans la réalisation de tâches de classification de textes.